# KorGPT2 챗봇

In [ ]:
!pip install transformers

## Tokenizer 선언

In [ ]:
from transformers import PreTrainedTokenizerFast

Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = "</s>"
EOS = "</s>"
MASK = "<unused0>"
SENT = "<unused1>"
PAD = "<pad>"

In [ ]:
# 허깅페이스 transformers에 등록된 pre-trained된 korGPT2 토크나이저를 가져옵니다
# PreTrainedTokenizerFast 클래스의 from_pretrained 메소드를 활용
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", \
                                                    bos_token=BOS, eos_token=EOS,\
                                                    unk_token="<unk>", pad_token=PAD,\
                                                    mask_token=MASK,)
tokenizer.tokenize("한국어 GPT-2를 활용해보겠습니다.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁한국어', '▁G', 'P', 'T', '-2', '를', '▁활용', '해보', '겠', '습니다.']

## 사전 학습된 korGPT2 언어 모델 객체 생성

In [ ]:
import torch
from transformers import GPT2LMHeadModel # LM Head가 추가된 GPT-2 모델 -> 생성형 NLP에서 쓰임

# GPT2LMHeadModel에서 from_pretrained 메소드로 사전 훈련된 GPT-2 모델을 로드
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

text = '딥러닝 개발을 잘 하기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')

gen_ids = model.generate(input_ids,
                         max_length=128,
                         repetition_penalty=2.0,
                         pad_token_id=tokenizer.pad_token_id,
                         eos_token_id=tokenizer.eos_token_id,
                         bos_token_id=tokenizer.bos_token_id,
                         use_cache=True)

generated = tokenizer.decode(gen_ids[0])
print(generated)



pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

딥러닝 개발을 잘 하기 위해서는 학습자의 뇌가 학습을 통해 어떤 정보를 얻어야 하는지 알아야 한다.
학습자는 자신의 두뇌를 어떻게 활용하느냐에 따라 그 정보가 얼마나 중요한지를 알 수 있다.
이런 점에서 볼 때 '뇌의 정보'는 매우 중요하다.
'정보'를 얻기 위해선 먼저 뇌의 기능을 이해하고 이를 바탕으로 한 훈련과 훈련을 해야 한다는 것이다.
그렇다면 이 두 가지 측면에서 보면 알파고는 과연 어느 정도의 수준의 지능인지 가늠할 수가 있을 것 같다.
알파고가 개발한 알고리즘은 다음과 같은 특징을 가지고 있는데, 우선, AI와 빅데이터 분석에 필요한 핵심 요소인 데이터 수집 능력을 향상시켰다.
AI는 데이터를 수집하고 분석하는 과정에서


## Chatbot Dataset을 다운로드 하기

dataset = 'https://github.com/songys/Chabot_data

In [ ]:
import pandas as pd
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename='ChatBotData.csv',
)

Chatbot_Data = pd.read_csv('ChatBotData.csv')

Chatbot_Data = Chatbot_Data[:1000]
Chatbot_Data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


## 데이터셋 생성

In [ ]:
from torch.utils.data import DataLoader, Dataset
import numpy as np
import re

# Chatbot Data를 처리하는 클래스 구현
class ChatbotDataset(Dataset):
  def __init__(self, chats, max_len=40): # 데이터셋 전처리
    self._data = chats
    self.max_len = max_len
    self.q_token = Q_TKN
    self.a_token = A_TKN
    self.sent_token = SENT
    self.eos = EOS
    self.mask = MASK
    self.tokenizer = tokenizer

  def __len__(self): # 챗봇 데이터의 길이를 리턴
    return len(self._data)

  def __getitem__(self, idx): # Pytorch의 Dataset class와 비슷한 역할.
  # 로드한 챗봇 데이터를 '차례차례' Dataloader에 넘겨주는 메소드
    turn = self._data.iloc[idx]
    q = turn["Q"] # 질문은 가져옵니다!
    q = re.sub(r"[?,!.]", r" ", q) # 구둣점 제거

    a = turn["A"]
    a = re.sub(r"[?,!.]", r" ", a)

    q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
    q_len = len(q_toked)

    a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
    a_len = len(a_toked)

    # mask = 질문길이 0 + 답변길이 1 + 나머지 0
    mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)

    # 답변 labels = [mask, mask, ..... mask, ..., <bos>, ..answer,... <eos>, <pad>.....]
    labels = [self.mask,] * q_len + a_toked[1:]

    # 답변 label을 고유 정수 인코딩
    label_ids = self.tokenizer.convert_tokens_to_ids(labels)

    # 최대 길이까지 PADDING
    while len(label_ids) < self.max_len:
      label_ids += [self.tokenizer.pad_token_id]

    # 질문 + 답변을 합쳐서 인덱스로 만듭니다
    token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)

    # 최대 길이까지 PADDING
    while len(token_ids) < self.max_len:
      token_ids += [self.tokenizer.pad_token_id]

    # 질문+답변, 마스크, 답변
    return (token_ids, np.array(mask), label_ids)


In [ ]:
# Helper function - batchwise
def collate_batch(batch):
  data = [item[0] for item in batch]
  mask = [item[1] for item in batch]
  label = [item[2] for item in batch]

  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
model.to(device)
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)


In [ ]:
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
import torch.nn as nn
import torch.optim as optim

lr = 3e-5
criterion = nn.CrossEntropyLoss(reduction='none')
optimizer = optim.Adam(model.parameters(), lr=lr)

epoch = 10
Sneg = -1e18
losses = 0

train_set = ChatbotDataset(Chatbot_Data, max_len=40)

train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0,\
                              shuffle=True, collate_fn=collate_batch)

print('START!')
for epoch in range(epoch):
  losses = 0
  # train dataloader에서 batchwise로 샘플을 가져와 학습 진행
  for batch_idx, samples in enumerate(train_dataloader):
    optimizer.zero_grad() # gradient를 0으로 초기화
    token_ids, mask, label = samples
    token_ids = token_ids.to(device)
    mask = mask.to(device)
    label = label.to(device)

    output = model(token_ids)
    out = output.logits # input element의 logit을 리턴

    # repeat_interleaver -> out의 2번째 차원의 크기만큼 반복
    mask_3d = mask.unsqueeze(dim=2).\
    repeat_interleave(repeats=out.shape[2], dim=2)

    # mask_3d가 1인 위치에는 출력값을 보존
    # mask_3d가 0인 위치에는 out과 같은 크기의 배열을 생성 후, 모든 element
    # 값을 Sneg으로 설정
    mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))

    # mask_out과 label로 손실 함수 계산
    loss = criterion(mask_out.transpose(2,1), label)

    # 평균 loss -> avg_loss[0] / avg_loss [1] -> 정규화
    avg_loss = loss.sum() / mask.sum()
    avg_loss.backward()

    optimizer.step()

    losses += avg_loss.item()
  print(f'epoch: %5d | loss: %.5f ' % (epoch+1, \
                                       losses/len(list(train_dataloader))))
print('END')



<ipython-input-14-24166bfb8522>:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


START!
epoch:     1 | loss: 35.86099 
epoch:     2 | loss: 34.59910 
epoch:     3 | loss: 33.87592 
epoch:     4 | loss: 33.49703 
epoch:     5 | loss: 33.27133 
epoch:     6 | loss: 32.92953 
epoch:     7 | loss: 33.12793 
epoch:     8 | loss: 32.89933 
epoch:     9 | loss: 33.08573 
epoch:    10 | loss: 33.08411 
END
